# 01. TensorFlow Lite를 활용한 AI 모델 배포

## 01-1. TensorFlow 개요

### TensorFlow란?
TensorFlow는 Google이 개발한 오픈소스 머신러닝 프레임워크로, 다양한 머신러닝 및 딥러닝 모델을 쉽게 구축하고 배포할 수 있도록 설계되었습니다. 강력한 연산 성능과 풍부한 라이브러리 생태계를 갖추고 있어 다양한 플랫폼에서 사용할 수 있습니다.

### TensorFlow의 특징
- **유연한 아키텍처**: 딥러닝 모델을 쉽게 구축하고 확장 가능
- **GPU 및 TPU 지원**: 강력한 연산 가속 기능 제공
- **풍부한 커뮤니티 지원**: 오픈소스로 개발되며 다양한 리소스 활용 가능
- **다양한 응용 분야**: 이미지 인식, 자연어 처리, 시계열 분석 등 다양한 머신러닝 작업에 활용

TensorFlow는 기본적으로 대규모 데이터를 처리하고 강력한 딥러닝 모델을 구축하는 데 유용하지만, 경량화된 모델을 필요로 하는 모바일 및 IoT 장치에서는 높은 연산 성능 요구로 인해 실행이 어렵습니다. 이를 해결하기 위해 등장한 것이 **TensorFlow Lite(TFLite)** 입니다.

## 01-2. TensorFlow Lite 개요

### TensorFlow Lite란?
TensorFlow Lite(TFLite)는 모바일 및 임베디드 디바이스에서 머신러닝 모델을 효율적으로 실행할 수 있도록 설계된 경량화된 프레임워크입니다. 기존 TensorFlow 모델을 변환하여 메모리 사용량을 줄이고 실행 속도를 최적화할 수 있습니다.

### TensorFlow Lite의 특징
- **경량화 모델**: 낮은 연산 성능을 가진 장치에서도 실행 가능
- **빠른 실행 속도**: 최적화된 커널을 사용하여 연산 속도를 향상
- **다양한 플랫폼 지원**: Android, iOS, Raspberry Pi 등에서 실행 가능
- **최적화된 모델 변환 지원**: 양자화(Quantization) 및 프루닝(Pruning) 적용 가능

## 01-3. Pre-trained 모델을 변환하여 활용하기
TFLite를 활용하여 사전 학습된 모델을 변환하고 Raspberry Pi에서 실행할 수 있습니다.

### 모델 변환 과정
1. TensorFlow에서 학습된 모델(.h5, .pb)을 준비
2. TFLite Converter를 사용하여 `.tflite` 파일로 변환
3. Raspberry Pi에서 TensorFlow Lite 인터프리터를 활용하여 모델 실행

# 02. [실습 4] TensorFlow Lite 모델 실행하기

## 02-1. TFLite MobileNet 모델을 다운로드하고 변환
**MobileNet**은 경량화된 CNN 기반 이미지 분류 모델로, 제한된 컴퓨팅 성능에서도 빠르게 실행됩니다.

### MobileNet TFLite 변환 과정
1. TensorFlow Hub에서 MobileNet 모델 다운로드
2. TensorFlow Lite Converter를 사용하여 `.tflite` 파일 변환
3. 변환된 모델을 Raspberry Pi로 전송

In [ ]:
import tensorflow as tf

# Pre-trained 모델 로드
model = tf.keras.applications.MobileNetV2(weights='imagenet')

# TFLite Converter 적용
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델 저장
with open('mobilenet_v2.tflite', 'wb') as f:
    f.write(tflite_model)

## 02-2. 이미지 입력 및 객체 분류 결과 출력
### 이미지 분류 테스트 코드
변환된 TFLite 모델을 사용하여 이미지를 입력하고 객체를 분류합니다.

In [ ]:
import tensorflow.lite as tflite
import numpy as np
from PIL import Image

# TFLite 모델 로드
interpreter = tflite.Interpreter(model_path='mobilenet_v2.tflite')
interpreter.allocate_tensors()

# 입력 및 출력 텐서 정보 가져오기
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# 이미지 전처리 및 입력
image = Image.open('test_image.jpg').resize((224, 224))
input_data = np.expand_dims(np.array(image, dtype=np.float32) / 255.0, axis=0)

interpreter.set_tensor(input_details[0]['index'], input_data)
interpreter.invoke()

# 예측 결과 출력
output_data = interpreter.get_tensor(output_details[0]['index'])
print("Prediction:", np.argmax(output_data))

## 02-3. Raspberry Pi에서 MobileNet 속도 테스트
TFLite 모델이 Raspberry Pi에서 얼마나 빠르게 실행되는지 테스트합니다.

In [ ]:
import time
start_time = time.time()
interpreter.invoke()
end_time = time.time()
print(f"Execution Time: {end_time - start_time:.4f} seconds")

# 03. 실시간 객체 탐지 개념

### 객체 탐지 모델의 원리 및 활용 방법
- **객체 탐지(Object Detection)**는 이미지에서 여러 객체를 찾아 분류하는 기술입니다.
- 두 가지 대표적인 기법:
  - **Bounding Box 기반 탐지** (YOLO, SSD 등)
  - **Segmentation 기반 탐지** (Mask R-CNN 등)

### MobileNet SSD vs YOLO 비교
| 모델 | 정확도 | 속도 | 경량화 |
|---|---|---|---|
| MobileNet SSD | 보통 | 빠름 | 높음 |
| YOLO | 높음 | 보통 | 중간 |

MobileNet SSD는 Raspberry Pi 같은 경량 장치에서 적합하며, YOLO는 높은 정확도가 필요한 경우 적합합니다.

# 04. [실습 5] 실시간 객체 탐지 모델 적용

## 04-1. COCO Dataset 기반 MobileNet SSD 모델 실행
MobileNet SSD를 활용하여 COCO 데이터셋 기반 객체 탐지를 실행합니다.

In [ ]:
import cv2
import tensorflow.lite as tflite

# TFLite 모델 로드
interpreter = tflite.Interpreter(model_path='ssd_mobilenet_v2.tflite')
interpreter.allocate_tensors()

# 카메라 영상 입력
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 객체 탐지 실행
    input_data = cv2.resize(frame, (300, 300))
    interpreter.set_tensor(input_details[0]['index'], [input_data])
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    
    # 화면 출력
    cv2.imshow('Object Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 04-2. 카메라 영상을 실시간으로 분석하고 객체 감지
카메라에서 실시간으로 데이터를 입력받아 객체를 탐지하는 모델을 실행합니다.

## 04-3. 감지된 객체에 따라 특정 액션 실행
객체가 감지되었을 때 Raspberry Pi의 GPIO를 활용하여 특정 액션(LED 점등, 비프음 출력)을 수행할 수 있습니다.

In [ ]:
import RPi.GPIO as GPIO
import time

# LED 및 부저 핀 설정
LED_PIN = 18
BUZZER_PIN = 23
GPIO.setmode(GPIO.BCM)
GPIO.setup(LED_PIN, GPIO.OUT)
GPIO.setup(BUZZER_PIN, GPIO.OUT)

# 객체 감지 시 실행할 동작
if detected_object == 'person':
    GPIO.output(LED_PIN, GPIO.HIGH)
    GPIO.output(BUZZER_PIN, GPIO.HIGH)
    time.sleep(1)
    GPIO.output(LED_PIN, GPIO.LOW)
    GPIO.output(BUZZER_PIN, GPIO.LOW)

GPIO.cleanup()